In [35]:
# preparing environment

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [36]:
# imports
from env import host, user, password
import pandas as pd
import numpy as np
import os

# create url to connect to database
def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
# pass sql query to database retrieve data
def new_zillow_data():
    sql_query = "SELECT * FROM properties_2017 JOIN predictions_2017 on predictions_2017.parcelid = properties_2017.parcelid WHERE unitcnt = 1 AND transactiondate BETWEEN '2017-05-01' AND '2017-06-30'"
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df.to_csv('zillow_df.csv')
    return df

In [37]:
df = new_zillow_data()

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13309 entries, 0 to 13308
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            13309 non-null  int64  
 1   parcelid                      13309 non-null  int64  
 2   airconditioningtypeid         6226 non-null   float64
 3   architecturalstyletypeid      0 non-null      object 
 4   basementsqft                  0 non-null      object 
 5   bathroomcnt                   13309 non-null  float64
 6   bedroomcnt                    13309 non-null  float64
 7   buildingclasstypeid           2 non-null      float64
 8   buildingqualitytypeid         13162 non-null  float64
 9   calculatedbathnbr             13296 non-null  float64
 10  decktypeid                    0 non-null      object 
 11  finishedfloor1squarefeet      0 non-null      object 
 12  calculatedfinishedsquarefeet  13308 non-null  float64
 13  f

dropping columns that meet any of the following criterion:
* more than 2,000 null values 
* do not contain relevant data for predicting property value 
    - examples include irrelevant IDs, fips, etc.
* taxamount, structuretaxvaluedollarcnt, landtaxvaluedollarcnt 
    - per project specifications we cannot use these columns
        - "for prediction purposes, we won't know tax amount until we know tax value"
    - confirmed by Data Science senior team member John Salas

In [34]:
df.nunique()

id                              13308
parcelid                        13308
bathroomcnt                        12
bedroomcnt                         12
buildingqualitytypeid              12
calculatedfinishedsquarefeet     3079
fips                                2
heatingorsystemtypeid               3
latitude                        12063
longitude                       11258
lotsizesquarefeet                8058
propertycountylandusecode          31
propertylandusetypeid               9
propertyzoningdesc               1212
rawcensustractandblock          10183
regionidcity                      123
regionidcounty                      2
regionidzip                       277
roomcnt                             1
unitcnt                             1
yearbuilt                         124
structuretaxvaluedollarcnt      10329
taxvaluedollarcnt               11074
assessmentyear                      1
landtaxvaluedollarcnt           10091
taxamount                       13212
censustracta

Dropping columns that meet any of the following criterion:
- categorical columns with more than 5 unique values 
    - it will be cumbersome to create a boolean value for each
- columns with only 1 value as 
    - our model won't be able to make any meaningful distinctions with them

While viewing data in SQL we noted several columns that appear to have identical data to one another. We will confirm if this is the case.

In [25]:
sqft_columns_diff = (df.finishedsquarefeet12 != df.calculatedfinishedsquarefeet).sum()

bathroom_count_diff = (df.calculatedbathnbr != df.bathroomcnt).sum()

bathroom_count_diff_alt = (df.fullbathcnt != df.bathroomcnt).sum()

print(sqft_columns_diff, bathroom_count_diff, bathroom_count_diff_alt)

13 13 13


dropping the following columns because with the exception of 13 rows, they are duplicates:
- finishedsquarefeet12 (majority duplicate of calculatedfinishedsquarefeet)
- calculatedbathnbr (majority duplicate of bathroomcnt)
- bathroomcnt (majority duplicate of bathroomcnt)

In [38]:
df = df.drop(columns=['airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft','yardbuildingsqft26', 'buildingclasstypeid', 'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'regionidneighborhood', 'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid', 'yardbuildingsqft17', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear'], axis=1)
# more than 2000 rows missing

df = df.drop(columns=['finishedsquarefeet12', 'calculatedbathnbr', 'fullbathcnt'], axis=1)
#dropping duplicate cols

df = df.drop(columns=['id', 'fips', 'parcelid', 'latitude', 'longitude', 'rawcensustractandblock', 'censustractandblock', 'heatingorsystemtypeid', 'regionidcounty', 'logerror' ], axis=1)
# irrelevant columns

df = df.drop(columns=['propertycountylandusecode', 'propertylandusetypeid','buildingqualitytypeid', 'propertyzoningdesc', 'regionidcity', 'regionidzip', 'yearbuilt', 'transactiondate' ], axis=1)
# categorical with too many values

df = df.drop(columns=['roomcnt', 'unitcnt', 'assessmentyear'], axis=1)
# only 1 value

df = df.drop(columns=['taxamount', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt'], axis=1)
# cannot use per instructions

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13309 entries, 0 to 13308
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   13309 non-null  float64
 1   bedroomcnt                    13309 non-null  float64
 2   calculatedfinishedsquarefeet  13308 non-null  float64
 3   lotsizesquarefeet             12976 non-null  float64
 4   taxvaluedollarcnt             13309 non-null  float64
dtypes: float64(5)
memory usage: 520.0 KB


We will change the names of our remaining columns to make them easier to understand
- Rename bathroomcnt to bathroom_count
- Rename bedroomcnt to bedroom_count
- Rename calculatedfinishedsquarefeet to unit_sq_feet
- Rename lotsizesquarefeet to lot_size_sq_feet
- Rename taxvaluedollarcnt to tax_dollar_value

lotsizesquarefeet has 333 missing values
- Out of 13309 rows, this is less than 3% of the total so we'll remove them along with the one null value row from calculatedfinishedsquarefeet

In [40]:
df.nunique()

bathroomcnt                        12
bedroomcnt                         12
calculatedfinishedsquarefeet     3079
lotsizesquarefeet                8058
taxvaluedollarcnt               11074
dtype: int64